1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД.<br>
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы.<br>
3. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

### Парсер с урока №2

In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bs
import numpy as np

In [2]:
def fix_salary(salary):
    
    drop = "./"
    
    before = ''
    after = ''
    min_salary = ''
    max_salary = ''
    
    for char in drop:
        salary = salary.replace(char,"")
 
    if 'от' in salary:
        after = 1   
    if 'до' in salary:
        before = 1
            
    if '–' in salary:
        min_salary, max_salary = salary.split("–")
    
    if after:
        min_salary = salary
    
    if before:
        max_salary = salary
        
    min_salary = re.sub(r'[^\x00-\x7F]+','', min_salary)
    max_salary = re.sub(r'[^\x00-\x7F]+','', max_salary)
    min_salary = min_salary.replace(' ', '')
    max_salary = max_salary.replace(' ', '')
                     
    salary = [min_salary, max_salary]
    
    return salary

def get_hh(text, area='1', salary=''):
    # text - название вакансии
    # area - расположение (id территории)
    # salary - зарплата
    
    url = 'https://hh.ru/search/vacancy'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
    }
    
    params = {'text': text}    
    
    if salary:
        params.update({'salary': salary, 'only_with_salary': 'true'}) 
     
    data = []
    
    html = requests.get(url, params=params, headers=headers)
    html = bs(html.text,'html.parser')
    
    pages = html.find('div', {'data-qa': 'pager-block'})
    
    if pages:
        last_page = int(pages.find_all('a', {'class': 'bloko-button'})[-2].getText())
    else:
        last_page = 1
    
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(url, params=params, headers=headers)
        html = bs(html.text,'html.parser')
        
        vacancy = html.find_all('div', {'class': 'vacancy-serp-item'})
        for item in vacancy:
            title = item.find('a', {'data-qa': 'vacancy-serp__vacancy-title'}).getText()
            salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
            salary = fix_salary(salary.get_text()) if salary else ['','']
            link = item.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']
            site = 'hh.ru'
            this = ({'title': title, 'min_salary': salary[0], 'max_salary': salary[1], 'link': link, 'site': site}) 
            data.append(this)
       
    return data

def get_sj(text, area='', salary=''):
    
    url = 'https://www.superjob.ru/vacancy/search'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
    }
    
    params = {'keywords': text}    
    if salary:
        params.update({'payment_value': salary, 'payment_defined': '1'}) 
     
    data = []
    
    html = requests.get(url, params=params, headers=headers)
    html = bs(html.text,'html.parser')
    
    pages = html.find('a', {'class': 'f-test-button-1'})
    
    if pages:
        pages = pages.findParent()
        last_page = int(pages.find_all('a')[-2].getText())
    else:
        last_page = 1
    
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(url, params=params, headers=headers)
        html = bs(html.text,'html.parser')
        
        vacancy = html.find_all('div', {'class': 'f-test-vacancy-item'})
        for item in vacancy:
            a = item.find_all('div')
            title = item.find('a', {'target': '_blank'}).getText()
            salary = item.find('span', {'class': 'f-test-text-company-item-salary'}).getText()
            site = 'superjob.ru'
            link = f"http://{site}{item.find('a', {'target': '_blank'})['href']}"
            
            salary = fix_salary(salary)
            this = ({'title': title, 'min_salary': salary[0], 'max_salary': salary[1], 'link': link, 'site': site}) 
            data.append(this)
            
        return data 

### Решение для урока №3

In [3]:
job = 'Программист' # Вакансия
by_salary = 200000 # Размер зп, по которому ищем

In [4]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['geek']

In [5]:
# Загружаем вакансии парсерами
all = get_hh(job) + get_sj(job)
df = pd.DataFrame(all)

In [6]:
db.collection.drop()
collection = db.lesson3

In [7]:
def add_job(df):
    a = 0
    b = 0
    for row in df.to_dict('records'):
        if row['min_salary']:
            row['min_salary'] ="".join(c for c in row['min_salary'] if c.isdigit())
            row['min_salary'] = int(row['min_salary'])
            
        if row['max_salary']:
            row['max_salary'] ="".join(c for c in row['max_salary'] if c.isdigit())
            row['max_salary'] = int(row['max_salary'])
        
        if db.collection.count_documents({'link': row['link']}) > 0:
            a += 1
        else:
            db.collection.insert_one(row)
            b += 1

    print(f'повторные вакансии, которые уже есть в БД: {a}, добавлено новых вакансий: {b}')

#### Загружаем вакансии в БД

In [8]:
add_job(df)

повторные вакансии, которые уже есть в БД: 79, добавлено новых вакансий: 741


#### Функция, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы.

In [9]:
def find_job(by_salary):
    res = db.collection.find( {
        '$or': [
            { 'min_salary': { '$gt': by_salary } }, { 'max_salary': { '$gt': by_salary } }
        ]
    } )
    
    return list(res)

In [10]:
target = find_job(by_salary)
target = pd.DataFrame(target)   
target = target.drop(columns=['_id'])

In [11]:
# Делаем кликабельными ссылки на вакансии
def make_clickable(val):
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)
target.style.format({'link': make_clickable})

,title,min_salary,max_salary,link,site
0,Senior PHP Developer / PHP программист,250000,350000,https://pravdinskiy.hh.ru/vacancy/49049468?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru
1,"Программист TypeScript, JS",170000,220000,https://pravdinskiy.hh.ru/vacancy/45342587?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru
2,Программист 1С для разработки своего решения,150000,250000,https://pravdinskiy.hh.ru/vacancy/49194647?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru
3,Старший программист-разработчик backend,250000,250000,https://pravdinskiy.hh.ru/vacancy/48881810?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru
4,Серверный разработчик / Backend Developer/Middle,150000,250000,https://pravdinskiy.hh.ru/vacancy/49133875?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru
5,Программист 1С,200000,250000,https://pravdinskiy.hh.ru/vacancy/48432485?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru
6,Ведущий программист,200000,300000,https://pravdinskiy.hh.ru/vacancy/49151839?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru
7,Senior PHP-разработчик (удаленно),300000,400000,https://pravdinskiy.hh.ru/vacancy/49201890?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru
8,Ведущий программист С++,150000,350000,https://pravdinskiy.hh.ru/vacancy/48724206?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru
9,Программист Ruby,120000,350000,https://pravdinskiy.hh.ru/vacancy/48574737?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru


### Решение для урока №8

1. В ранее написанное приложение добавить класс с функциями, которые позволят собрать открытые данные по выбранной теме при помощи Python с сайта (выберите из списка известных источников данных).

Суть:
Сравнить зарплату найденных вакансий со сведениями о доходах государственных гражданских служащих ФАС России и подведомственных учреждений. Открытые данные на сайте Минфина: https://minfin.gov.ru/ru/opendata/?opendata_q_4=%D0%B4%D0%BE%D1%85%D0%BE%D0%B4

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
class GovEarnings():
    def __init__(self):
        self.url = 'http://fas.gov.ru/opendata/7703516539-gains/data-20190717T0000-structure-20170711T0000.csv'
        self.df = pd.read_csv(self.url,sep=',')
        self.clear_data()
    
    def clear_data(self):
        searchfor = ['Несовершеннолетний ребенок', 'Супруга', 'Cупруг', '-', '(супруга)']
        self.df = self.df[~self.df['name'].str.contains('|'.join(searchfor))]
        self.df = self.df[['earnings']]
        self.df = self.df[(self.df['earnings'] != '-') & (self.df['earnings'] != '0')] 
        self.df['earnings'] = self.df['earnings'].str.replace(',', '.')
        self.df['earnings'] = self.df['earnings'].astype(float)
        
    def get_min(self):
        return int(round(self.df['earnings'].min()/12, 0))
    def get_max(self):
        return int(round(self.df['earnings'].max()/12, 0))
    def get_mean(self):
        return int(round(self.df['earnings'].mean()/12, 0))

In [14]:
ministry = GovEarnings()

In [15]:
target.loc[target['min_salary'] == "", 'min_salary'] = 0
target.loc[target['max_salary'] == "", 'max_salary'] = 0
target.loc[target['min_salary'].isna(), 'min_salary'] = 0
target.loc[target['max_salary'].isna(), 'max_salary'] = 0

In [16]:
target['salary'] = target['max_salary']
target.loc[target['salary'] == 0, 'salary'] = target['min_salary']

In [17]:
target['to_min_gov'] = (target['salary']*100/ministry.get_min()).astype('float').round(0).astype('int').astype('str')+'%'
target['to_mean_gov'] = (target['salary']*100/ministry.get_mean()).astype('float').round(0).astype('int').astype('str')+'%'
target['to_max_gov'] = (target['salary']*100/ministry.get_max()).astype('float').round(0).astype('int').astype('str')+'%'

In [18]:
target.style.format({'link': make_clickable})

,title,min_salary,max_salary,link,site,salary,to_min_gov,to_mean_gov,to_max_gov
0,Senior PHP Developer / PHP программист,250000,350000,https://pravdinskiy.hh.ru/vacancy/49049468?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru,350000,10526%,223%,10%
1,"Программист TypeScript, JS",170000,220000,https://pravdinskiy.hh.ru/vacancy/45342587?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru,220000,6617%,140%,6%
2,Программист 1С для разработки своего решения,150000,250000,https://pravdinskiy.hh.ru/vacancy/49194647?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru,250000,7519%,160%,7%
3,Старший программист-разработчик backend,250000,250000,https://pravdinskiy.hh.ru/vacancy/48881810?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru,250000,7519%,160%,7%
4,Серверный разработчик / Backend Developer/Middle,150000,250000,https://pravdinskiy.hh.ru/vacancy/49133875?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru,250000,7519%,160%,7%
5,Программист 1С,200000,250000,https://pravdinskiy.hh.ru/vacancy/48432485?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru,250000,7519%,160%,7%
6,Ведущий программист,200000,300000,https://pravdinskiy.hh.ru/vacancy/49151839?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru,300000,9023%,192%,9%
7,Senior PHP-разработчик (удаленно),300000,400000,https://pravdinskiy.hh.ru/vacancy/49201890?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru,400000,12030%,255%,11%
8,Ведущий программист С++,150000,350000,https://pravdinskiy.hh.ru/vacancy/48724206?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru,350000,10526%,223%,10%
9,Программист Ruby,120000,350000,https://pravdinskiy.hh.ru/vacancy/48574737?from=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82,hh.ru,350000,10526%,223%,10%
